<a href="https://colab.research.google.com/github/Nicolas-Ayala-Collazos/ML/blob/Version-4/Algoritmo_de_Decaimiento_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import networkx as nx
import geopandas as gpd
import folium
import pandas as pd
#from pyvis.network import Network
# Cargar el shapefile

# Carga aquí tu DataFrame df_incentivos
df_incentivos = pd.read_csv('/content/INCENTIVOS.csv', encoding='ISO-8859-1')

# Crear el mapeo de códigos a nombres y de nombres a códigos
mapeo_codigos_nombres = dict(zip(df_incentivos['CODIGO'], df_incentivos['NOMBRE']))
mapeo_nombres_codigos = {nombre: codigo for codigo, nombre in mapeo_codigos_nombres.items()}

shapefile_path = '/content/MAYA.shp'  # Reemplaza con la ruta correcta a tu archivo shapefile
shapefile = gpd.read_file(shapefile_path)

# Crear el grafo a partir del shapefile
G = nx.Graph()
for segment in shapefile['LINE_SEGME']:
    nodes = segment.split('-')[:2]
    nodes = [node[:-2] if node.endswith('-1') else node for node in nodes]
    G.add_edge(nodes[0], nodes[1])


for nodo in G.nodes():
    G.nodes[nodo]['nombre'] = mapeo_codigos_nombres.get(nodo, nodo)
# Aquí generamos el layout de Kamada-Kawai
pos_kamada_kawai = nx.kamada_kawai_layout(G)


# Filtrar los nodos del grafo
nodos_cd102 = [nodo for nodo in G.nodes() if nodo.startswith('CD10.2.')]

# Mapear estos nodos con df_incentivos
nodos_info = df_incentivos[df_incentivos['CODIGO'].isin(nodos_cd102)]

# Calcular sumatorias y conteo
suma_clientes_cd102 = nodos_info['CLIENTES'].sum()
suma_capacidad_cd102 = nodos_info['CAPACIDAD'].sum()
conteo_nodos_cd102 = len(nodos_cd102)

# Mostrar los resultados
print(f"Suma de Clientes (): {suma_clientes_cd102}")
print(f"Suma de Capacidad (): {suma_capacidad_cd102}")
print(f"Conteo de Nodos (): {conteo_nodos_cd102}")


def explorar_ramales_del_camino(nodo_inicio, nodo_fin, grafo):
    nodos_explorados = set()
    aristas_exploradas = set()

    # Exploración en profundidad desde cada nodo en el camino
    def explorar_desde_nodo(nodo_actual, visitados):
        if nodo_actual == nodo_fin:
            return
        for vecino in grafo.neighbors(nodo_actual):
            if vecino not in visitados and vecino != nodo_fin:
                nodos_explorados.add(vecino)
                aristas_exploradas.add((nodo_actual, vecino))
                visitados.add(vecino)
                explorar_desde_nodo(vecino, visitados)

    # Iniciar la exploración desde el nodo inicial
    explorar_desde_nodo(nodo_inicio, set([nodo_inicio]))

    return nodos_explorados, aristas_exploradas
# Definir la función de barrido eléctrico extendido

def obtener_nodos_y_aristas_en_camino(nodo_inicio, nodo_fin, grafo):
    if nodo_inicio not in grafo or nodo_fin not in grafo:
        return set(), set()

    try:
        camino = nx.shortest_path(grafo, nodo_inicio, nodo_fin)
    except nx.NetworkXNoPath:
        return set(), set()

    nodos_en_camino = set(camino)
    aristas_en_camino = set()

    for i in range(len(camino) - 1):
        aristas_en_camino.add((camino[i], camino[i + 1]))

    # Agregar nodos adyacentes al camino
    for nodo in camino:
        for vecino in grafo.neighbors(nodo):
            nodos_en_camino.add(vecino)
            aristas_en_camino.add((nodo, vecino))
            # Explorar vecinos de los vecinos
            for vecino_del_vecino in grafo.neighbors(vecino):
                if vecino_del_vecino not in nodos_en_camino:
                    nodos_en_camino.add(vecino_del_vecino)
                    aristas_en_camino.add((vecino, vecino_del_vecino))

    return nodos_en_camino, aristas_en_camino


# Función de barrido eléctrico extendido
def barrido_electrico_extendido(nodo_inicio, nodo_fin, grafo):
    if nodo_inicio not in grafo or nodo_fin not in grafo:
        return "Uno o ambos nodos no están en el grafo", set(), set()

    try:
        camino = nx.shortest_path(grafo, nodo_inicio, nodo_fin)
    except nx.NetworkXNoPath:
        return "No hay un camino entre los nodos.", set(), set()

    nodos_conectados, aristas_conectadas = obtener_nodos_y_aristas_en_camino(nodo_inicio, nodo_fin, grafo)
    return camino, nodos_conectados, aristas_conectadas

# Crear Combobox widgets para la entrada de texto con sugerencias de nodos
#lista_nodos = list(G.nodes)
#nodo_inicio_input = widgets.Combobox(options=lista_nodos, description='Nodo Inicio:', ensure_option=True, disabled=False)
#nodo_fin_input = widgets.Combobox(options=lista_nodos, description='Nodo Fin:', ensure_option=True, disabled=False)

# Crear Combobox widgets para la entrada de texto con sugerencias de nodos
lista_nombres_nodos = [G.nodes[nodo]['nombre'] for nodo in G.nodes() if nodo in mapeo_codigos_nombres]

nodo_inicio_input = widgets.Combobox(options=lista_nombres_nodos, description='Nodo Inicio:', ensure_option=True, disabled=False)
nodo_fin_input = widgets.Combobox(options=lista_nombres_nodos, description='Nodo Fin:', ensure_option=True, disabled=False)

# Botón para ejecutar el barrido eléctrico
boton_barrido = widgets.Button(description='Realizar Barrido')

# Salida para mostrar los resultados
salida_barrido = widgets.Output()

def oscurecer_color(color):
    """Devuelve una versión más oscura del color dado."""
    colores = {
        'green': 'darkgreen',
        'red': 'gray',
        'cyan': 'white',
        'orange': 'darkorange',
        'gray': 'dimgray',
        'lime': 'limegreen',
        'black': 'black'
    }
    return colores.get(color, color)

caminos_acumulados = []
nodos_conectados_acumulados = set()

def on_boton_barrido_clicked(b):
    global caminos_acumulados, nodos_conectados_acumulados, aguas_abajo, aguas_arriba
    # Limpiar la salida anterior
    salida_barrido.clear_output()

    # Obtener los nodos de los widgets
    # Convertir nombres a códigos originales
    nodo_inicio = mapeo_nombres_codigos.get(nodo_inicio_input.value, nodo_inicio_input.value)
    nodo_fin = mapeo_nombres_codigos.get(nodo_fin_input.value, nodo_fin_input.value)

    # Realizar el barrido eléctrico extendido
    camino, nodos_conectados, aristas_conectadas = barrido_electrico_extendido(nodo_inicio, nodo_fin, G)
       # Acumular los resultados
    caminos_acumulados.append(camino)
    nodos_conectados_acumulados.update(nodos_conectados)

# Función para obtener coordenadas de un nodo
    def obtener_coordenadas_nodo(nodo, shapefile):
        fila_nodo = shapefile[shapefile['LINE_SEGME'].str.contains(nodo)]
        if not fila_nodo.empty:
            return fila_nodo.iloc[0].geometry.coords[0]
        return None
    # Crear un mapa base
    centroid = shapefile.geometry.centroid
    map_location = [centroid.y.mean(), centroid.x.mean()]
    m = folium.Map(location=map_location, zoom_start=16)

        # Agregar marcador para el nodo de inicio


    for _, row in shapefile.iterrows():
        line_points = [[point[1], point[0]] for point in row['geometry'].coords]
        folium.PolyLine(line_points, color="black", weight=2, opacity=0.5).add_to(m)



    # Función para obtener el color basado en el prefijo del nodo
    def obtener_color_nodo(nodo):
        if nodo.startswith('CD10.2.'):
            return 'red'
        elif nodo.startswith('CD10.4.'):
            return 'cyan'
        else:
            return 'green'  # Color por defecto

    # Dibujar los caminos del barrido en un color diferente
    for camino in caminos_acumulados:
        for i in range(len(camino) - 1):
            coord_inicio = obtener_coordenadas_nodo(camino[i], shapefile)
            coord_fin = obtener_coordenadas_nodo(camino[i + 1], shapefile)
            if coord_inicio and coord_fin:
                folium.PolyLine([coord_inicio, coord_fin], color="blue", weight=3, opacity=1).add_to(m)

    # Dibujar los Nodos Conectados Acumulados con información de df_incentivos
    # Dibujar los nodos en el mapa
    # Dibujar solo los nodos que comienzan con 'CD10.2.' en el mapa
    for nodo in nodos_conectados_acumulados:
        if nodo.startswith('CD10.2.'):
            nodo_info = df_incentivos[df_incentivos['CODIGO'] == nodo]
            if not nodo_info.empty:
                lat = nodo_info.iloc[0]['LATITUD']
                lon = nodo_info.iloc[0]['LONGITUD']
                etiqueta = f"CABINA: {nodo_info.iloc[0]['CABINAS']}, CLIENTES: {nodo_info.iloc[0]['CLIENTES']}, POTENCIA: {nodo_info.iloc[0]['CAPACIDAD']}"

                folium.Marker([lat, lon], popup=etiqueta, icon=folium.Icon(color='red')).add_to(m)

        # Agregar un marcador de color naranja para el nodo CD00.1.
    # Agregar un marcador de color naranja para el nodo CD00.1. desde el shapefile
    # Iterar sobre el shapefile y colocar marcadores para los nodos que comienzan con 'CD00.1.'
    for _, row in shapefile.iterrows():
        # Supongamos que los nodos están en la columna 'LINE_SEGME'
        nodos_segmento = row['LINE_SEGME'].split('-')[:2]
        for nodo in nodos_segmento:
            if nodo.startswith('CD00.1.'):
                coord_nodo = obtener_coordenadas_nodo(nodo, shapefile)
                if coord_nodo:
                    # No hay necesidad de mapeo aquí, simplemente colocar el marcador
                    folium.Marker(
                        location=[coord_nodo[1], coord_nodo[0]],
                        icon=folium.Icon(color='orange')
                    ).add_to(m)




    # Agregar marcador para el nodo de inicio
    coord_nodo_inicio = obtener_coordenadas_nodo(nodo_inicio, shapefile)
    if coord_nodo_inicio:
        nombre_nodo_inicio = mapeo_codigos_nombres.get(nodo_inicio, nodo_inicio)
        folium.Marker(
            location=[coord_nodo_inicio[1], coord_nodo_inicio[0]],
            popup=f"Nodo Inicio: {nombre_nodo_inicio}",
            icon=folium.Icon(color='green', icon='info-sign')
        ).add_to(m)


    # Dibujar los nodos conectados
    for nodo in nodos_conectados_acumulados:
        coord_nodo = obtener_coordenadas_nodo(nodo, shapefile)
        if coord_nodo:
            folium.CircleMarker(
                location=[coord_nodo[1], coord_nodo[0]],
                radius=5,
                color='blue',
                fill=True,
                popup=nodo
            ).add_to(m)

    # Mostrar los resultados y el grafo
    with salida_barrido:
        #print("Camino Actual:", camino)
        #print("Todos los Caminos:", caminos_acumulados)
        #print("Nodos Conectados Acumulados:", nodos_conectados_acumulados)

        # Asignar colores a los nodos según sus prefijos y destacar el nodo de inicio y fin
        node_colors = []
        for node in G.nodes():
            color_base = 'gray'  # Color por defecto
            if node.startswith('CD00.1.'):
                color_base = 'orange'
            elif node.startswith('CD10.3.'):
                color_base = 'green'
            elif node.startswith('CD10.2.'):
                color_base = 'red'
            elif node.startswith('CD10.4.'):
                color_base = 'cyan'

            # Oscurecer los nodos conectados al camino
            if node in nodos_conectados_acumulados:
                color = oscurecer_color(color_base)
            else:
                color = color_base

            # Colores especiales para nodo de inicio y fin
            if node == nodo_inicio:
                color = 'lime'
            elif node == nodo_fin:
                color = 'black'

            node_colors.append(color)

        # Configurar el layout con spring_layout
        pos = nx.kamada_kawai_layout(G)

         # Ajusta el valor de 'k' según sea necesario

        # Dibujar el grafo con este layout
        plt.figure(figsize=(20, 20))
        nx.draw(G, pos, edge_color="black", node_color=node_colors, font_size=8, node_size=40)
        # Dibujar las etiquetas de los nodos
        etiquetas_nodos = nx.get_node_attributes(G, 'nombre')
        nx.draw_networkx_labels(G, pos, labels=etiquetas_nodos, font_size=5, font_weight='bold')

        plt.show()
                # Mostrar el mapa
        display(m)

                # Filtrar nodos_conectados_acumulados para obtener solo aquellos que comienzan con 'CD10.2.'
        nodos_filtrados_cd102 = [nodo for nodo in nodos_conectados_acumulados if nodo.startswith('CD10.2.')]

        # Mapear estos nodos al DataFrame df_incentivos
        nodos_info = df_incentivos[df_incentivos['CODIGO'].isin(nodos_filtrados_cd102)]

        # Actualizar la variable global

        mapeo_nodos_cd102 = df_incentivos[df_incentivos['CODIGO'].isin(nodos_filtrados_cd102)][['CODIGO', 'CABINAS']]
        aguas_abajo = mapeo_nodos_cd102.to_dict('records')

        caminos_cd103 = [camino for camino in caminos_acumulados if any(nodo.startswith('CD10.3.') for nodo in camino)]
        nodos_cd103 = set(nodo for camino in caminos_cd103 for nodo in camino)
        mapeo_caminos_cd103 = df_incentivos[df_incentivos['CODIGO'].isin(nodos_cd103)][['CODIGO', 'CABINAS']]

        # Actualizar la variable global
        aguas_arriba = mapeo_caminos_cd103.to_dict('records')


        # Calcular sumatorias y conteo
        suma_clientes = nodos_info['CLIENTES'].sum()
        suma_capacidad = nodos_info['CAPACIDAD'].sum()
        conteo_nodos = len(nodos_filtrados_cd102)

        # Guardar la información en un arreglo
        INFORMACION = [suma_clientes, suma_capacidad, conteo_nodos]

        # Mostrar el resultado
        print(f"Clientes: {suma_clientes}, Capacidad: {suma_capacidad}, Cantidad: {conteo_nodos}")
        print(f"TOTAL de Clientes (): {suma_clientes_cd102}, TOTAL de Capacidad (): {suma_capacidad_cd102}, TOTAL de Cabinas (): {conteo_nodos_cd102}")

    # Mostrar los resultados y el grafo

# Asignar la acción al botón
boton_barrido.on_click(on_boton_barrido_clicked)

# Mostrar los widgets
display(nodo_inicio_input, nodo_fin_input, boton_barrido, salida_barrido)





Suma de Clientes (): 3408.0
Suma de Capacidad (): 4743.0
Conteo de Nodos (): 160


Combobox(value='', description='Nodo Inicio:', ensure_option=True, options=('D.CA1986', 'D.CA2012', 'D.CA04051…

Combobox(value='', description='Nodo Fin:', ensure_option=True, options=('D.CA1986', 'D.CA2012', 'D.CA04051', …

Button(description='Realizar Barrido', style=ButtonStyle())

Output()

In [ ]:
aguas_arriba